# Inteface to run the geopoints utility

This is a demo interface that enables running of the geopoints function in the utilities module

In [1]:
from reAnalysisInterface import demoInterface
d = demoInterface()

utilities:Xarray Version = 2022.3.0


HTML(value='\n            <style>\n                /* enlarges the default jupyter cell outputs, can revert by…